In [1]:
from astropy.table import Table, join
import numpy as np
from astropy.coordinates import SkyCoord, match_coordinates_sky


In [2]:
photom_file = 'CEERS_v0.51.3_photom.fits'
photz_file = 'CEERS_v0.51.3_photz_quantities.fits'
galfit_file = 'ceers_v0.51_galfit.fits'
barro_file = 'ceers_all_v0.51_eazy.hdf5'
seplusfile = 'ceers_f200w_fmf_merged.fits'
# vetted_z5_file = 'CEERS_v0.51_z4.5to5.5.cat'
# vetted_z7_file = 'CEERS_v0.51_z5.5to8.5.cat'
# vetted_z_gt8_file = 'CEERS_v0.51_z8.5to14_renamed.txt'

# Read the general catalogs
cat = Table.read(photom_file)
photz = Table.read(photz_file)
galfit = Table.read(galfit_file)
barro = Table.read(barro_file)
seplus = Table.read(seplusfile)

In [5]:
galfit.columns

<TableColumns names=('ID','FIELD','CATID','RA','DEC','F200W_MAG','F200W_MAG_ERR','F200W_RE','F200W_RE_ERR','F200W_N','F200W_N_ERR','F200W_Q','F200W_Q_ERR','F200W_PA','F200W_PA_ERR','F200W_FLAG','F356W_MAG','F356W_MAG_ERR','F356W_RE','F356W_RE_ERR','F356W_N','F356W_N_ERR','F356W_Q','F356W_Q_ERR','F356W_PA','F356W_PA_ERR','F356W_FLAG')>

In [4]:
# Read the general catalogs
cat = Table.read(photom_file)
photz = Table.read(photz_file)
galfit = Table.read(galfit_file)
barro = Table.read(barro_file)
seplus = Table.read(seplusfile)

def AB(nJy):
    return -2.5 * np.log10(nJy) + 31.4

# Add columns for ab mag and S/N
bands = ['115','150','200','277','356','410','444','606','814','105','125','160']
for b in bands:
    cat[f"AB{b}"] = AB(cat[f"FLUX_{b}"])
    cat[f"SN{b}"] = cat[f"FLUX_{b}"]/cat[f"FLUXERR_{b}"]

# Add columns for probability integral in dz=1 intervals
# Add ID column to photoz catalog
zl = [3,4,5,6,7,8,9,10,11,12,13,14]
for z in zl:
    photz[f"INT_PZ{z}_{z+1}"] = photz[f"INT_ZGT{z}"] - photz[f"INT_ZGT{z+1}"]
photz['INT_PZ8_10'] = photz["INT_ZGT8"] - photz["INT_ZGT10"]
photz['ID'] = np.arange(len(photz))+1
photz['Z68_range'] = photz['ZU68'] - photz['ZL68']

# Add selection columns to barro catalog
barro['z4-5'] = (barro['z160'] > 4.0) & (barro['z840']<=5.0)
barro['z5-6'] = (barro['z160'] > 5.0) & (barro['z840']<=6.0)
barro['z6-7'] = (barro['z160'] > 6.0) & (barro['z840']<=7.0)
barro['z7-8'] = (barro['z160'] > 7.0) & (barro['z840']<=8.0)
barro['z8-10'] = (barro['z160'] > 8.0) & (barro['z840']<=10.0)
barro['zGT10'] = (barro['z160'] > 10.0)
barro['zphot_range'] = barro['z840'] - barro['z160']

# prepend an identifier on the columns of catalogs that Steve didn't make
for c in galfit.colnames:
    if c != 'ID':
        galfit.rename_column(c,f"galfit_{c}")
for c in barro.colnames:
    if c != 'ID':
        barro.rename_column(c,f"barro_{c}")
for c in seplus.colnames:
    if c != 'ID':
        seplus.rename_column(c,f"seplus_{c}")

# Join the catalogs
j1 = join(cat,photz,keys='ID')
j2 = join(j1,galfit,keys='ID')
j3 = join(j2,barro,keys='ID')

# Position-match to SE++ catalog
j3['coords'] = SkyCoord(j3['RA'],j3['DEC'],unit=('deg','deg'),frame='icrs')
seplus['coords'] = SkyCoord(seplus['seplus_world_centroid_alpha'],
                            seplus['seplus_world_centroid_delta'],unit=('deg','deg'),frame='icrs')
# idx are the indices in seplus that are the closest match to each source in j3
idx, d2d, d3d = match_coordinates_sky(j3['coords'], seplus['coords']) 
j3['seplus_idx'] = idx
j3['seplus_sep'] = d2d.to('arcsec')

# Now remove the skycoord column, because doesn't save to HDF5 (I think)
j3.remove_column('coords')

/var/folders/hf/dh49zz3n1x5d1p_b4qn8g4z00000gn/T/ipykernel_66218/4068906225.py:18: RuntimeWarning: invalid value encountered in log10
  return -2.5 * np.log10(nJy) + 31.4
/var/folders/hf/dh49zz3n1x5d1p_b4qn8g4z00000gn/T/ipykernel_66218/4068906225.py:18: RuntimeWarning: divide by zero encountered in log10
  return -2.5 * np.log10(nJy) + 31.4
       'AB ZP:        31.40', 'Library:      Bruzual & Charlot (2003)',
       'SFH:          Exponentially declining SFH: SFR ~ exp(-t/tau)',
       'Stellar IMF:  Chabrier', 'metallicity:  0.020',
       'log(tau/yr):  8.3    - 10.0, in steps of 0.20',
       'log(age/yr):  7.6    - 10.1, in steps of 0.10',
       'A_V:          0.0    -  4.0, in steps of 0.10',
       'z:            0.0100 -  8.0000, in steps of 0.0100',
       'Filters:     364 365 366 375 376 383 377   4   6 203 204 205',
       'ltau: log[tau/yr], lage: log[age/yr], lmass: log[mass/Msol], lsfr: log[sfr/(Msol/yr)], lssfr: log[ssfr*yr], la2t: log[age/tau]',
       'For sfr=0. ls

In [9]:
# Write out the files
j3.write('merged_v0.51_catalog.fits')#,serialize_meta=True,overwrite=True)


OSError: File merged_v0.51_catalog.fits already exists. If you mean to replace it then use the argument "overwrite=True".